### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [12]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [14]:
user_mat, movie_mat = FunkSVD(ratings_mat, 4, 0.005, 10)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 44.809344
2 		 16.974322
3 		 4.855348
4 		 2.994176
5 		 2.781023
6 		 2.718815
7 		 2.674870
8 		 2.632008
9 		 2.586048
10 		 2.535250


In [15]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.99863537   9.19370664  10.70021667   9.91221582]
 [  8.61402685   7.36617958   9.31457854   8.63915731]
 [  7.75964806   7.07650806   8.31364594   7.6903623 ]
 [  9.61514159   8.11348321   9.96086861   9.70899772]
 [  8.5118183    7.5562975    8.91089904   8.51559151]
 [  6.76981201   5.76212006   7.48990168   6.79691741]
 [  9.32528849   8.27185547   9.31800652   9.30045465]
 [  8.32524011   7.21250356   8.34781983   8.31754811]
 [  8.50827395   7.38096887   8.95881979   8.45519466]
 [  7.618678     6.71553956   8.20531158   7.58331747]
 [  9.00525918   7.65827189   9.09170062   9.04923452]
 [  9.47915843   8.45425489  10.26586568   9.42484772]
 [  9.29158425   8.05681462   9.70149858   9.28902896]
 [  6.65912374   5.65937932   6.93672718   6.70591841]
 [  9.71936475   8.28882314  10.28236008   9.84191931]
 [  9.80189073   8.62206795  10.03562744   9.81977893]
 [  8.31049954   7.7133365    8.54072401   8.14620679]
 [  7.3981454    6.43988102   6.76673864   7.34098994]
 [  8.1574

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [16]:
user_mat, movie_mat = FunkSVD(ratings_mat, 4, 0.005, 250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 40.383177
2 		 14.248444
3 		 4.340356
4 		 2.929642
5 		 2.741946
6 		 2.686232
7 		 2.652518
8 		 2.622616
9 		 2.592130
10 		 2.559711
11 		 2.524821
12 		 2.487205
13 		 2.446746
14 		 2.403438
15 		 2.357369
16 		 2.308711
17 		 2.257706
18 		 2.204640
19 		 2.149810
20 		 2.093491
21 		 2.035891
22 		 1.977111
23 		 1.917119
24 		 1.855726
25 		 1.792582
26 		 1.727192
27 		 1.658949
28 		 1.587183
29 		 1.511230
30 		 1.430516
31 		 1.344655
32 		 1.253548
33 		 1.157481
34 		 1.057206
35 		 0.953971
36 		 0.849497
37 		 0.745876
38 		 0.645392
39 		 0.550291
40 		 0.462538
41 		 0.383606
42 		 0.314356
43 		 0.255007
44 		 0.205213
45 		 0.164203
46 		 0.130942
47 		 0.104287
48 		 0.083110
49 		 0.066377
50 		 0.053189
51 		 0.042798
52 		 0.034597
53 		 0.028104
54 		 0.022941
55 		 0.018816
56 		 0.015504
57 		 0.012831
58 		 0.010663
59 		 0.008895
60 		 0.007447
61 		 0.006256
62 		 0.005272
63 		 0.004456
64 		

In [17]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.42515079   9.9999999    9.99999997  10.00000008]
 [  9.99999991   3.99999989   8.99999996  10.00000014]
 [  7.99999956   8.99999958   9.99999986   5.0000006 ]
 [  8.99999962   7.99999967   9.99999989  10.0000005 ]
 [  9.99999951   4.99999955   8.99999985   9.00000067]
 [  5.99999974   3.99999978   9.99999994   6.00000035]
 [  9.00000027   8.00000028  10.00000012   8.99999964]
 [ 10.00000017   5.00000016   9.00000007   7.99999979]
 [  7.00000007   8.00000007  10.00000004   7.99999992]
 [  9.00000023   5.00000022   9.00000008   6.9999997 ]
 [  9.00000018   8.00000018  10.00000007   7.99999976]
 [  9.00000006  10.00000009  10.00000003   8.9999999 ]
 [ 10.00000006   9.00000005  10.00000002   7.99999993]
 [  5.00000008   8.00000008   5.00000003   7.99999989]
 [ 10.00000047   8.00000044  10.00000015   9.99999937]
 [  8.99999975   8.99999978   9.99999991  10.00000032]
 [  9.00000042   8.0000004    8.00000013   7.99999943]
 [  9.99999981   7.99999983   0.99999994  10.00000023]
 [  5.0000

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [18]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [19]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, 4, 0.005, 250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 39.228077
2 		 12.565884
3 		 4.014281
4 		 2.987134
5 		 2.833552
6 		 2.788550
7 		 2.767357
8 		 2.751305
9 		 2.735686
10 		 2.719138
11 		 2.701041
12 		 2.680909
13 		 2.658249
14 		 2.632529
15 		 2.603176
16 		 2.569566
17 		 2.531037
18 		 2.486893
19 		 2.436423
20 		 2.378931
21 		 2.313770
22 		 2.240401
23 		 2.158459
24 		 2.067826
25 		 1.968711
26 		 1.861720
27 		 1.747902
28 		 1.628753
29 		 1.506162
30 		 1.382304
31 		 1.259476
32 		 1.139898
33 		 1.025533
34 		 0.917929
35 		 0.818142
36 		 0.726722
37 		 0.643772
38 		 0.569044
39 		 0.502056
40 		 0.442194
41 		 0.388802
42 		 0.341237
43 		 0.298905
44 		 0.261271
45 		 0.227864
46 		 0.198268
47 		 0.172112
48 		 0.149065
49 		 0.128823
50 		 0.111106
51 		 0.095654
52 		 0.082226
53 		 0.070595
54 		 0.060552
55 		 0.051906
56 		 0.044479
57 		 0.038114
58 		 0.032668
59 		 0.028013
60 		 0.024039
61 		 0.020647
62 		 0.017752
63 		 0.015281
64 		

In [20]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.18563496959474:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [21]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, 4, 0.005, 20)#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.298298
2 		 10.829729
3 		 7.417788
4 		 5.711942
5 		 4.635632
6 		 3.883611
7 		 3.325062
8 		 2.893012
9 		 2.549404
10 		 2.270730
11 		 2.041419
12 		 1.850586
13 		 1.690315
14 		 1.554684
15 		 1.439147
16 		 1.340141
17 		 1.254822
18 		 1.180891
19 		 1.116466
20 		 1.060000


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [22]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [23]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
